In [1]:
import torch 
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import math

from datasets import load_dataset
dataset = load_dataset("gigaword")

Using custom data configuration default
Reusing dataset gigaword (/tmp/xdg-cache/huggingface/datasets/gigaword/default/1.2.0/c518c578e42a6afe842b09e979ee2907ea42a12b57ba992fae9e9d7347825245)


In [2]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
src_text = dataset['train']['document'][0:1000]
target_text = dataset['train']['summary'][0:1000]
model_name = 'google/pegasus-gigaword'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)

train_data = tokenizer.prepare_seq2seq_batch(src_text, target_text, return_tensors="pt", truncation="only_first", padding="longest", max_length=64)

input_ids_train = train_data['input_ids']
attention_masks_train = train_data['attention_mask']
labels_train = train_data['labels']

In [3]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, PegasusConfig, modeling_outputs

In [5]:
config = PegasusConfig.from_pretrained(model_name, output_hidden_states=True, output_attentions=True)

In [6]:
pega_copyback_model = PegasusForConditionalGeneration.from_pretrained(model_name, config=config).to(torch_device)

Some weights of PegasusGuidedCopyBack were not initialized from the model checkpoint at google/pegasus-gigaword and are newly initialized: ['outdegree_score_w', 'indegree_score_w', 'p_gen_w.weight', 'p_gen_w.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

batch_size = 2
dataloader_train = DataLoader(dataset_train,sampler=RandomSampler(dataset_train), batch_size=batch_size)

#freezing the parameters
# for param in pega_copyback_model.model.parameters():
#     param.requires_grad = False
optimizer = AdamW(filter(lambda p: p.requires_grad, pega_copyback_model.parameters()),lr=5e-5)
                  
epochs = 5

In [8]:
from tqdm.notebook import tqdm
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)  

for epoch in tqdm(range(1, epochs+1)):
    
    pega_copyback_model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for b in progress_bar:

        optimizer.zero_grad()
        
        b = tuple(x.to(torch_device) for x in b)
        
        inputs = {'input_ids':      b[0],
                  'attention_mask': b[1],
                  'labels':         b[2],
                 }       

        outputs = pega_copyback_model(**inputs)
#         vocab_size =  outputs['logits'].shape[2]
    
        loss = outputs[0]#criterion(outputs.view(-1,vocab_size),b[2].view(-1))

        loss_train_total += loss.item()
        loss.backward()

#         torch.nn.utils.clip_grad_norm_(pega_copyback_model.parameters(), 1.0)

        optimizer.step()
        #scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(b))})
         
        
    torch.save(pega_copyback_model.state_dict(), f'data/finetuned_pega_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')

/home/esen/.local/lib/python3.7/site-packages/torch/nn/functional.py:1958: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."



Epoch 1
Training loss: 5.9626273908615115



Epoch 2
Training loss: 0.8384230832308531



Epoch 3
Training loss: 0.6054406896829605



Epoch 4
Training loss: 0.4688126783259213



Epoch 5
Training loss: 0.34567064413055776



Epoch 6
Training loss: 0.26391870155371727


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-08387f1d2b40>", line 38, in <module>
    optimizer.step()
  File "/home/esen/.local/lib/python3.7/site-packages/transformers/optimization.py", line 304, in step
    p.data.addcdiv_(exp_avg, denom, value=-step_size)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1148, in get_records
    return 

KeyboardInterrupt: 

In [10]:
pega_copyback_model.eval()

tgt_text = tokenizer.batch_decode(pega_copyback_model.generate(train_data['input_ids'][[0],:].to(torch_device)))#, skip_special_tokens=True)

In [11]:
print(tgt_text)

['australian current current account current account current account narrows #.## bln dlrs in june quarter on commodity price rise narrows to #.unk_3']


In [12]:
from datasets import load_metric

pega_copyback_model.eval()


metric = load_metric("rouge")

src_text = dataset['validation']['document'][0:100]
target_text = dataset['validation']['summary'][0:100]

val_data = tokenizer.prepare_seq2seq_batch(src_text, target_text, return_tensors="pt", truncation="only_first", padding="longest", max_length=64)

input_ids_val = val_data['input_ids']
attention_masks_val = val_data['attention_mask']
labels_val = val_data['labels']

dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

batch_size = 1
dataloader_val = DataLoader(dataset_val, batch_size=batch_size)


shouldPrint = 0
toPrint = 4
with torch.no_grad():
    for b in dataloader_val:        
        b = tuple(x.to(torch_device) for x in b)
        
        inputs = {'input_ids':      b[0],
                  'attention_mask': b[1],
                  'labels':         b[2],
                 }       
        gen = tokenizer.batch_decode(pega_copyback_model.generate(inputs['input_ids']), skip_special_tokens=True)
        if shouldPrint < toPrint:
            print("Source Sentence:", src_text[shouldPrint])
            print("Sample Sentence:", gen[0])
            shouldPrint += 1 
        ref = tokenizer.batch_decode(inputs['labels'])
        metric.add_batch(predictions=gen, references=ref)


print("@@@@@@@@@@@@@@@@@@@ Rogue @@@@@@@@@@@@@@@@@@@@@@@")
print()        

print(metric.compute())

{'rouge1': AggregateScore(low=Score(precision=0.1554928400944054, recall=0.4795815746753247, fmeasure=0.23136775513804625), mid=Score(precision=0.17402431231662066, recall=0.5282606421356422, fmeasure=0.2569692341725045), high=Score(precision=0.19291390263704156, recall=0.579778896103896, fmeasure=0.2824507151708098)), 'rouge2': AggregateScore(low=Score(precision=0.06152143285722438, recall=0.2030489538239538, fmeasure=0.09302674511225241), mid=Score(precision=0.07464696147510039, recall=0.24736994949494936, fmeasure=0.11211079923104261), high=Score(precision=0.0904801216513209, recall=0.29384617604617597, fmeasure=0.13413271281551845)), 'rougeL': AggregateScore(low=Score(precision=0.14158101591581065, recall=0.43408622835497856, fmeasure=0.21143435889558784), mid=Score(precision=0.1583793080593422, recall=0.48157323232323257, fmeasure=0.23368659118166588), high=Score(precision=0.17625338100710924, recall=0.5238439213564217, fmeasure=0.25546915132337167)), 'rougeLsum': AggregateScore(l